***LIBRERIAS NECESARIAS***

keras/tensorflow para cargar y ejecutar nuestro modelo

Opencv para el procesamiento de video

In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import imutils
import time
import cv2
import os

***Funcion para detectar tapabocas en imagen (frame)***

- Se ingresa el frame a evaliar
- detecta rostro en la imagen : Para este caso se usa una red ya entrena para detectar rostros
- de aplica el detector entrenado de tapabocas 

In [2]:
def detect_tapabocas(frame, faceNet, maskNet):
    #preprocesamiento realizado con la funcio blob de opencv para cambiar el tamaño del frame
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300),
        (104.0, 177.0, 123.0))

    # pasamos la imagen pre-procesada por el face_detector
    faceNet.setInput(blob)
    detections = faceNet.forward()
    
    faces = []
    locs = []
    prediccion = []

    # recorremos las faces detectadas
    for i in range(0, detections.shape[2]):
        # extraemos el cofidence
        confidence = detections[0, 0, i, 2]

        # miramos que supere un umbral minimo
        if confidence > 0.5:
            # extraemos las coordenadas de la caja (box
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # Se asegura que la caja este en las dimensiones del marco
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))

            # extraemos la imagen del rostro, cortando la original con las medidas de la box
            #se le aplica el preprosamiento para entrar al modelo detector de tapabocas
            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)
            face = np.expand_dims(face, axis=0)

            # adicionamos face y la caja en un lista c/u
            faces.append(face)
            locs.append((startX, startY, endX, endY))

    # se aplica el predictor se se detecto algun rostro
    if len(faces) > 0:
        
        prediccion = maskNet.predict(faces)

    # retornamos la prediccion y la caja para luego dibujarla
    return (locs, prediccion)

In [3]:
#modelos para face detection
protoPath= r'.\face_detector\deploy.prototxt'
pesosPath=r'.\face_detector\res10_300x300_ssd_iter_140000.caffemodel'

#modelo para mask_detection
maskPath=r'model_detector2.model'

In [4]:
faceNet= cv2.dnn.readNet(protoPath, pesosPath)

In [5]:
maskNet= load_model(maskPath)

In [7]:
#inicializo la camara de video
vs = VideoStream(src=0).start()
time.sleep(2.0)

while True:
    # cambio el tamaño de la camara a un maximo de 400 pixeles
    frame = vs.read()
    frame = imutils.resize(frame, width=500)

    # detecto rostros en el frame y si tienen tapabocas o no
    (locs, preds) = detect_tapabocas(frame, faceNet, maskNet)

    # ciclo para dibujar la caja y la prediccion de mask
    for (box, pred) in zip(locs, preds):
        
        (startX, startY, endX, endY) = box
        (mask, withoutMask) = pred

        # determino si tiene o no mascara y el color de la letra
        label = "Loser_with_mask" if mask > withoutMask else "Loser"
        color = (0, 255, 0) if label == "Loser_with_mask" else (0, 0, 255)

        # incluyo la probabilidad de tener tapabocas
        label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

        # muestro el frame
        cv2.putText(frame, label, (startX, startY - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

    # show the output frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    if key == ord("q"):
        break

# do a bit of cleanup
cv2.destroyAllWindows()
vs.stop()

error: OpenCV(4.2.0) ..\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
